<a href="https://colab.research.google.com/github/gagan3012/keytotext/blob/master/notebooks/Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keytotext --upgrade
!pip install datasets

     |████████████████████████████████| 1.2MB 10.5MB/s 
     |████████████████████████████████| 1.8MB 47.1MB/s 
     |████████████████████████████████| 808kB 41.3MB/s 
     |████████████████████████████████| 2.3MB 48.2MB/s 
     |████████████████████████████████| 174kB 51.9MB/s 
     |████████████████████████████████| 102kB 12.5MB/s 
     |████████████████████████████████| 133kB 51.4MB/s 
     |████████████████████████████████| 112kB 51.6MB/s 
     |████████████████████████████████| 276kB 51.3MB/s 
     |████████████████████████████████| 10.6MB 48.5MB/s 
     |████████████████████████████████| 829kB 40.1MB/s 
     |████████████████████████████████| 3.3MB 36.6MB/s 
     |████████████████████████████████| 901kB 38.1MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
     |████████████████████████████████| 1.3MB 42.9MB/s 
     |████████████████████████████████| 143kB 54.7MB/s 
     |████████████████████████████████| 296kB 53.6MB/s 
  Created wheel for pathtools: filename=pathtool

In [2]:
from datasets import load_dataset

dataset = load_dataset('common_gen',split='train')

Using custom data configuration default


Dataset common_gen downloaded and prepared to /root/.cache/huggingface/datasets/common_gen/default/2020.5.30/1a9e8bdc026c41ce7a9e96260debf7d2809cb7fd63fa02b017e4fac1b00c6b23. Subsequent calls will reuse this data.


In [3]:
import pandas as pd

df = pd.DataFrame()
df['keywords']=dataset['concepts']
df['text'] = dataset['target']
df.head()

,keywords,text
0,"[ski, mountain, skier]",Skier skis down the mountain
1,"[ski, mountain, skier]",A skier is skiing down a mountain.
2,"[ski, mountain, skier]",Three skiers are skiing on a snowy mountain.
3,"[wag, tail, dog]",The dog is wagging his tail.
4,"[wag, tail, dog]",A dog wags his tail at the boy.


In [4]:
def clean(keywords):
  source_text = ' '.join(map(str, keywords))
  return source_text

In [5]:
df['keywords'] = df['keywords'].apply(lambda x: clean(x))
df.head()

,keywords,text
0,ski mountain skier,Skier skis down the mountain
1,ski mountain skier,A skier is skiing down a mountain.
2,ski mountain skier,Three skiers are skiing on a snowy mountain.
3,wag tail dog,The dog is wagging his tail.
4,wag tail dog,A dog wags his tail at the boy.


In [6]:
from keytotext import KeytotextTrainer

model = KeytotextTrainer()
model.from_pretrained(model_name="t5-small")
model.train(data_df=df[:1000],batch_size=4, max_epochs=3, use_gpu=True)
model.save_model()

Global seed set to 42


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc



  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Global seed set to 42


In [8]:
model.load_model("/content/outputs/keytotext-epoch-3", use_gpu=True)

keywords=["ski", "mountain", "sky"]
model.predict(keywords)

'a sky on the mountain of skis.'